In [3]:
import pandas as pd 
import matplotlib.pyplot as plt
from prophet import Prophet
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np
import matplotlib.pyplot as plt

In [12]:
data_path="D:\OLAEM stock\OLAE Historical Data.csv"
df=pd.read_csv(data_path)
df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,02-01-2026,40.48,37.89,40.74,37.80,199.21M,7.89%
1,01-01-2026,37.52,36.30,38.16,36.04,85.39M,3.53%
2,31-12-2025,36.24,36.80,37.50,35.91,48.24M,-0.93%
3,30-12-2025,36.58,35.60,36.80,35.33,48.54M,3.25%
4,29-12-2025,35.43,36.37,36.43,34.85,37.84M,-2.10%


In [13]:
missing_values=df.isnull().sum()
statistics=df.describe()
missing_values, statistics

(Date        0
 Price       0
 Open        0
 High        0
 Low         0
 Vol.        0
 Change %    0
 dtype: int64,
             Price        Open        High         Low
 count  312.000000  312.000000  312.000000  312.000000
 mean    58.298590   58.574904   59.974167   57.103462
 std     17.358625   17.518843   17.966320   16.963926
 min     31.260000   32.010000   33.440000   30.760000
 25%     44.700000   45.387500   46.412500   43.570000
 50%     52.850000   52.890000   54.330000   51.865000
 75%     70.372500   71.010000   72.990000   69.550000
 max     99.260000  100.000000  102.500000   99.000000)

In [11]:
# First, let's check the current column names and data
display("Column names:", df.columns.tolist())
display("First few rows:", df.head())

# Now let's fix the column names by stripping whitespace
df.columns = df.columns.str.strip()
display("\nAfter stripping whitespace from column names:", df.columns.tolist())

# Now the rest of your code should work
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y', errors='coerce')
df.set_index('Date', inplace=True)

# Create a figure and a set of subplots
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(15, 10))

# Plot the Open price over time
df['Open'].plot(ax=ax[0], color='blue')
ax[0].set_title('Open Price Over Time')
ax[0].set_ylabel('Price ($)')

# Plot the trading volume over time
# Clean the 'Vol.' column (remove 'M' and convert to float)
df['Volume'] = df['Vol.'].str.replace('M', '').astype(float)
df['Volume'].plot(ax=ax[1], color='green')
ax[1].set_title('Trading Volume Over Time')
ax[1].set_ylabel('Volume (Millions)')

# Improve layout
plt.tight_layout()
plt.show()

'Column names:'

['Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']

'First few rows:'

,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
2026-01-02,40.48,37.89,40.74,37.80,199.21M,7.89%
2026-01-01,37.52,36.30,38.16,36.04,85.39M,3.53%
2025-12-31,36.24,36.80,37.50,35.91,48.24M,-0.93%
2025-12-30,36.58,35.60,36.80,35.33,48.54M,3.25%
2025-12-29,35.43,36.37,36.43,34.85,37.84M,-2.10%


'\nAfter stripping whitespace from column names:'

['Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']

KeyError: 'Date'